# Modelo de Recomendacion (Similitud de Coseno)

la propuesta para el sistema de recomendacion es aplicar un modelo que debera tener una relacion item-item, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicar la similitud del coseno.

In [1]:
# importamos librerias
import pandas as pd 

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# del dataframe steam_games extraemos solamente las columnas que vamos a necesitar
steam_games= pd.read_parquet('data/clean_steam_games.parquet')
modelo_recomendacion=steam_games[['item_id', 'app_name', 'genres']]
modelo_recomendacion.head()

,item_id,app_name,genres
0,761140,Lost Summoner Kitty,Action
1,761140,Lost Summoner Kitty,Casual
2,761140,Lost Summoner Kitty,Indie
3,761140,Lost Summoner Kitty,Simulation
4,761140,Lost Summoner Kitty,Strategy


In [3]:
# creamos los dummies
modelo_recomendacion= pd.get_dummies(modelo_recomendacion, columns=['genres'], prefix='')
modelo_recomendacion=modelo_recomendacion.groupby(['item_id','app_name']).sum().reset_index()

modelo_recomendacion.head()

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counterstrike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day Of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,Halflife Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


aplicamos similitud de coseno

In [4]:
similitudes= cosine_similarity(modelo_recomendacion.iloc[:,3:])

c:\Users\bryam\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [5]:
print(similitudes)

[[1.         1.         1.         ... 1.         0.70710678 1.        ]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 ...
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 [0.70710678 0.70710678 0.70710678 ... 0.70710678 1.         0.70710678]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]]


In [6]:
similitudes.shape

(27458, 27458)

In [7]:
def recomendacion_juego(id):
    
    id = int(id)
    # filtramos el juego e igualarlo a  su ID
    juego_seleccionado = modelo_recomendacion[modelo_recomendacion['item_id'] == id]
    # devolver error en caso de vacio
    if juego_seleccionado.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # calculamos la matriz de similitud coseno
    #similitudes = cosine_similarity(modelo_item.iloc[:,3:])
    
    # calcualmos la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similitudes[modelo_recomendacion[modelo_recomendacion['item_id'] == id].index[0]]
    
    # calculamos los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # obtenemos los nombres de los juegos 5 recomendados
    juegos_recomendados = modelo_recomendacion.iloc[indices_juegos_similares]['app_name']
    
    return juegos_recomendados

In [8]:
recomendacion_juego(761140)

22933                            Mind Maze
19487    Desolate Wastes Vendor Chronicles
14416              War Birds Ww Air Strike
9464         World Of Cinema Directors Cut
25252                       Wooden Battles
Name: app_name, dtype: string

In [9]:
modelo_recomendacion

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counterstrike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day Of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,Halflife Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27453,2028055,Tom Clancys Ghost Recon Future Soldier Season ...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27454,2028056,Worms Revolution Season Pass,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
27455,2028062,Call Of Duty Black Ops Ii Season Pass,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27456,2028103,Assassins Creed Iii Season Pass,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


se divide a la mitad el dataframe a fin de lograr que se pueda montar el modelo en render

In [10]:
# contamos filas
cantidad_filas=len(modelo_recomendacion)
# calculamos la mitad
mitad_filas= cantidad_filas // 10
# seleccionamos la mitad superior
modelo_render= modelo_recomendacion.iloc[:mitad_filas]

In [11]:
modelo_render.shape

(2745, 24)

In [12]:
similitudes_render= cosine_similarity(modelo_render.iloc[:,3:])

almacenamos dataframe modelo_render

In [13]:
modelo_render.to_parquet('data/modelo_render.parquet')

probamos la funcion otra vez

generamos una nueva funcion que solo recorra las similaridades con la fila del juego que se le ingresa como parametro, para que el archivo sea corrido por render

In [14]:
def encontrar_juegos_similares(id):
    # encontramos el indice del juego ingresado por el id
    juego_indice = modelo_render.index[modelo_render['item_id'] == id].tolist()[0]
    
    # extraemos las caracteristicas del juego ingresado
    juego_caracteristicas = modelo_render.iloc[juego_indice, 3:].values.reshape(1, -1)
    
    #calculamos la similitud de coseno entre el juego ingresado y los demas juegos
    similitudes_render = cosine_similarity(modelo_render.iloc[:, 3:], juego_caracteristicas)
    
    # obtenemos los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similitudes_render.argsort(axis=0)[::-1][1:6]
    indices_juegos_similares = indices_juegos_similares.flatten()[1:]
    
    # obtenemos los juegos más similares en función de los índices
    juegos_similares = modelo_render.iloc[indices_juegos_similares]['app_name']
    
    return juegos_similares

In [15]:
encontrar_juegos_similares(3310)

435    Samantha Swift And The Hidden Roses Of Athena
434              Discovery A Seek And Find Adventure
433                                            Luxor
432                                      Little Farm
Name: app_name, dtype: string

generamos la funcion para obtener recomendaciones

In [22]:
def recomendacion_juego(id: int):
    
    # verificamos si el juego con game_id existe en el dataframe
    game = modelo_render[modelo_render['item_id'] == id]

    if game.empty:
        return("El juego '{id}' no posee registros.")
    
    # obtenemos el indice del juego dado
    idx = game.index[0]

    # tomamos una muestra aleatoria del dataframe
    sample_size= 2000  # definimos el tamaño de la muestra (ajusta según sea necesario)
    df_sample = modelo_render.sample(n=sample_size, random_state=42)  # ajustamos la semilla aleatoria segun sea necesario

    # calculamos la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([modelo_render.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # obtenemos las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # ordenamos los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # obtenemos los 5 juegos mas similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # lista de los juegos similares (solo nombres)
    similar_game_names = df_sample['app_name'].iloc[similar_game_indices].tolist()

    return {"similar_games": similar_game_names}

In [33]:
recomendacion_juego(10)

{'similar_games': ['Star Wars Jedi Knight Mysteries Of The Sith',
  'Mafia Ii Dlc Greaser Pack',
  'Saints Row The Third Witches Wieners Pack',
  'Call Of Duty Modern Warfare Collection',
  'Battlestations Midway']}